### Barcode Demultiplexer Simulator 

- Simulate Demultiplexing using local and semi-global alignment

In [ ]:
import sys
sys.path.append("/home/emre/github_repo/MinION")
from minION import analyser
from minION import consensus
from minION import demultiplexer
import importlib
from minION.util import IO_processor
from minION.util.globals import BARCODES, MEDAKA_MODELS, DEFAULT_TARGETS
importlib.reload(IO_processor)
importlib.reload(analyser)
importlib.reload(consensus)
importlib.reload(demultiplexer)
from pathlib import Path
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
#import plotly.express as px
import random
from Bio import SeqIO
import subprocess
import re
import shutil

In [ ]:
# Plotly version
def barcode_score(summary):
    fig = px.histogram(summary, x="barcode_score", nbins=100, 
                       title="Barcode Score Distribution", labels={'barcode_score': 'Barcode Score', 'count': 'Frequency'})
    fig.update_traces(marker_color='red', marker_line_color='black', marker_line_width=1.5, opacity=0.5)
    fig.update_layout(width=600, height=400)
    return fig

def barcode_barplot(summary, barcode_id="barcode_arrangement", ylim=None):
    barcodes = summary[barcode_id].value_counts().reset_index()
    fig = px.bar(barcodes, x=barcode_id, y="count", 
                 title="Barcode Frequency", labels={'index': 'Barcode ID', barcode_id: 'Frequency'})
    fig.update_traces(marker_color='red', marker_line_color='black', marker_line_width=1.5, opacity=0.5)
    fig.update_layout(width=600, height=400)

    if ylim:
        fig.update_layout(yaxis=dict(range=[0, ylim]))

    return fig

def barcode_arrangement(summary, plot=True):
    barcode_arrangements = summary["barcode_arrangement"].value_counts().reset_index()

    if plot:
        fig = px.bar(barcode_arrangements, x='barcode_score', y='barcode_arrangement', 
                     title="Barcode Frequency", labels={'index': 'Barcode ID', 'barcode_arrangement': 'Frequency'})
        fig.update_traces(marker_color='red', marker_line_color='black', marker_line_width=1.5, opacity=0.5)
        fig.update_layout(width=600, height=400)
        return fig
    else:
        return barcode_arrangements

In [ ]:

def check_gc_content(sequence, desired_gc_fraction, tolerance=0.05):
    """
    Check if the GC content of the sequence is within the desired range.
    """
    gc_count = sum(1 for nucleotide in sequence if nucleotide in ['G', 'C'])
    gc_fraction = gc_count / len(sequence)
    return abs(gc_fraction - desired_gc_fraction) <= tolerance

def generate_gc_content_sequence(length, desired_gc_fraction):
    """
    Iteratively generate a random DNA sequence until it has the desired GC content.
    """
    while True:
        sequence = ''.join(random.choice('ATCG') for _ in range(length))
        if check_gc_content(sequence, desired_gc_fraction):
            return sequence


def introduce_mutations(sequence, mutation_rate, weights = [0.2, 0.4, 0.4]):
    """
    Introduce mutations in a sequence with the specified mutation rate.
    The mutations include substitution, insertion, and deletion.
    """
    mutated_sequence = ''
    nucleotides = ['A', 'C', 'G', 'T']
    
    weights = {'substitution': weights[0], 'insertion': weights[1], 'deletion': weights[2]}

    i = 0
    while i < len(sequence):
        if random.random() < mutation_rate:
            mutation_type = random.choices(['substitution', 'insertion', 'deletion'], weights=[weights['substitution'], weights['insertion'], weights['deletion']])[0]
            if mutation_type == 'substitution':
                mutated_sequence += random.choice([n for n in nucleotides if n != sequence[i]])
                i += 1
            elif mutation_type == 'insertion':
                mutated_sequence += random.choice(nucleotides)
            else:  # deletion
                i += 1
                continue
        else:
            mutated_sequence += sequence[i]
            i += 1
    return mutated_sequence

### Generate the synthetic sequences

- GC content of 60 %


In [ ]:

def write_to_fastq(filename, barcodes, sequence_length, barcode_position_start, barcode_position_end, mutation_rate, num_sequences, num_noise):
    """
    Write mutated sequences for each barcode into a FASTQ file using a fixed surrounding sequence.
    """
    with open(filename, 'wt') as fastq_file:
        for i in range(num_sequences):
            # Generate one surrounding sequence for the set of barcodes
            surrounding_sequence = generate_gc_content_sequence(sequence_length, 0.6)

            for barcode_name, barcode_sequence in barcodes.items():
                # Introduce mutations in the barcode sequence
                mutated_barcode_sequence = introduce_mutations(barcode_sequence, mutation_rate)

                # Insert the mutated barcode sequence in the surrounding sequence
                mutated_sequence = surrounding_sequence[:barcode_position_start] + mutated_barcode_sequence + surrounding_sequence[barcode_position_end:]
                
                # Assign quality scores
                quality_scores = "I" * len(mutated_sequence)
                
                mut_r = str(mutation_rate).replace(".", "_")
                # Write to FASTQ file
                fastq_file.write(f"@seq_{mut_r}_{i + 1}_{barcode_name}\n")
                fastq_file.write(f"{mutated_sequence}\n")
                fastq_file.write("+\n")
                fastq_file.write(f"{quality_scores}\n")


        for i in range(num_noise):
        # Generate random noise sequence
            noise_sequence = generate_gc_content_sequence(sequence_length, 0.6)
            
            # Assign quality scores
            quality_scores = "I" * sequence_length
            
            # Write to FASTQ file
            fastq_file.write(f"@noise_{i + 1}\n")
            fastq_file.write(f"{noise_sequence}\n")
            fastq_file.write("+\n")
            fastq_file.write(f"{quality_scores}\n")

def split_read_id(read_id):
    """
    Split the read ID into the barcode ID and the read ID.
    """
    parts = read_id.split("_")
    barcode_id = parts[-1]
    read_id_prefix = "_".join(parts[:-1])

    if barcode_id.isnumeric():
        barcode_id = "noise"
    return barcode_id



In [ ]:
from Bio import SeqIO
barcodes = {}
with open("/home/emre/github_repo/MinION/minION/barcoding/minion_barcodes_sim.fasta", "r") as handle:
    records = list(SeqIO.parse(handle, "fasta"))
    for record in records:
        barcodes[record.id] = str(record.seq) 

In [ ]:


names = ["0", "01", "02", "03", "04"]
for i, rate in enumerate([0, 0.1, 0.2, 0.3, 0.4]):
    sequence_length = 150
    barcode_position_start = 70
    barcode_position_end = 94
    mutation_rate = rate # 10% mutation rate
    num_sequences = 9000
    num_noise = 0

    # Write sequences to a FASTQ file
    write_to_fastq(f"data/mutated_sequences_{names[i]}_8barcodes.fastq", barcodes, sequence_length, barcode_position_start, barcode_position_end, mutation_rate, num_sequences, num_noise)


In [ ]:
#file_names = ["00_0", "00_1", "00_2", "01_0", "01_1", "01_2", "02_0", "02_1", "02_2"]
file_names = ["tpr_fpr_testing"]

for file in file_names:
    path = Path("data") / file / "8_barcodes" 
    demultiplexer.run_demultiplexer_single(path, BARCODES, 15, 15, basecall_folder = path)

    # Calculate TP, FP, FN, TN
    #summary = pd.read_csv(Path("data") / file / "24_barcodes" / "demultiplex_15" / "barcoding_summary.csv")
    #summary["Truth"] = summary["read_id"].apply(split_read_id)
    

### CPP Analyser

In [ ]:
### Summary for CPP 

summary = pd.read_csv("/home/emre/github_repo/MinION/examples/data/Demultiplex_cpp/barcoding_summary.txt", sep="\t")
#summary = pd.read_csv("/home/emre/github_repo/MinION/examples/data/tpr_fpr_testing/24_barcodes/demultiplex_15/barcoding_summary.txt", sep="\t")
summary["Truth"] = summary["ID"].apply(split_read_id)
bins = np.arange(15, 105, 5)
summary["bins"] = pd.cut(summary["RBC_Score"], bins)

# Set the aesthetic style of the plots
sns.set_theme(style="whitegrid")

# Create figure and axes
fig, ax = plt.subplots(figsize=(10, 5))

# Create the histogram plot
sns.histplot(data=summary, x="RBC_Score", bins=100, kde=False, ax=ax, color="navy", alpha=0.5)


# Add custom legend with the correct colors
custom_lines = [plt.Line2D([0], [0], color="navy", lw=4)]

#ax.legend(custom_lines, [''])
# Add labels and title
ax.set_xlabel("Barcode Score")
ax.set_ylabel("Frequency")
#ax.set_title("Barcode Score Distribution")

# Add a red vertical line
#ax.axvline(x=60, color='red', linestyle='--')

# Show the plot
plt.savefig("barcode_score_distribution_guppy_quality_test_8barcodes_CPP.png", dpi=300)
plt.show()

In [ ]:
TPR_interval = {"Interval" : [], "TPR" : [], "FPR" : []}
barcode_column = "RBC"
for intervals in summary["bins"].unique().sort_values():
    bin_df = summary[summary["bins"] == intervals]
    
    tpr = []
    fpr = []

    fp = bin_df[(bin_df[barcode_column] != bin_df["Truth"])]
    tp = bin_df[(bin_df[barcode_column] == bin_df["Truth"])]

    print(f"TPR for {intervals}: {len(tp) / (len(bin_df) +1)}")
    print(f"FPR for {intervals}: {len(fp) / (len(bin_df)+1)}")

    TPR_interval["Interval"].append(intervals)
    TPR_interval["TPR"].append(len(tp) / (len(bin_df) +1))
    TPR_interval["FPR"].append(len(fp) / (len(bin_df)+1))

pd.DataFrame(TPR_interval).to_csv("tpr_fpr_8barcodes_CPP.csv", index=False)
pd.DataFrame(TPR_interval).dropna().plot(x="Interval", y=["TPR", "FPR"], figsize=(10, 5), title="TPR and FPR for different barcode score intervals")

In [ ]:
summary = pd.read_csv("/home/emre/github_repo/MinION/examples/data/tpr_fpr_testing/24_barcodes/demultiplex_15/barcoding_summary.txt", sep="\t")
summary

### Guppy Analyser

In [ ]:
summary = pd.read_csv("/home/emre/github_repo/MinION/examples/data/tpr_fpr_testing/24_barcodes/demultiplex_15/barcoding_summary.txt", sep="\t")
summary["Truth"] = summary["read_id"].apply(split_read_id)
bins = np.arange(15, 105, 5)
summary["bins"] = pd.cut(summary["barcode_score"], bins)

# Set the aesthetic style of the plots
sns.set_theme(style="whitegrid")

# Create figure and axes
fig, ax = plt.subplots(figsize=(10, 5))

# Create the histogram plot
sns.histplot(data=summary, x="barcode_score", bins=100, kde=False, ax=ax, color="navy", alpha=0.5)


# Add custom legend with the correct colors
custom_lines = [plt.Line2D([0], [0], color="navy", lw=4)]

#ax.legend(custom_lines, [''])
# Add labels and title
ax.set_xlabel("Barcode Score")
ax.set_ylabel("Frequency")
#ax.set_title("Barcode Score Distribution")

# Add a red vertical line
#ax.axvline(x=60, color='red', linestyle='--')

# Show the plot
plt.savefig("barcode_score_distribution_guppy_quality_test_24barcodes.png", dpi=300)
plt.show()



In [ ]:
TPR_interval = {"Interval" : [], "TPR" : [], "FPR" : []}
barcode_column = "barcode_full_arrangement"
for intervals in summary["bins"].unique().sort_values():
    bin_df = summary[summary["bins"] == intervals]
    
    tpr = []
    fpr = []

    fp = bin_df[(bin_df[barcode_column] != bin_df["Truth"])]
    tp = bin_df[(bin_df[barcode_column] == bin_df["Truth"])]

    print(f"TPR for {intervals}: {len(tp) / (len(bin_df) +1)}")
    print(f"FPR for {intervals}: {len(fp) / (len(bin_df)+1)}")

    TPR_interval["Interval"].append(intervals)
    TPR_interval["TPR"].append(len(tp) / (len(bin_df) +1))
    TPR_interval["FPR"].append(len(fp) / (len(bin_df)+1))


    pd.DataFrame(TPR_interval).to_csv("tpr_fpr_8barcodes.csv", index=False)


In [ ]:
summary.groupby(["Truth", "bins"]).count()["read_id"].reset_index().pivot(index="bins", columns="Truth", values="read_id").plot.bar(stacked=True, figsize=(10, 5))


In [ ]:
# Get TP and FP
barcode_column = "barcode_full_arrangement"
barcode = "RB01"
tp = summary[(summary[barcode_column] == barcode) & (summary["Truth"] == barcode)]

### TPR FPR Plotting of Guppy and Local Alignment

In [ ]:
barcode_8_cpp = pd.read_csv("/home/emre/github_repo/MinION/examples/tpr_fpr_8barcodes_CPP.csv")
barcode_24_cpp = pd.read_csv("/home/emre/github_repo/MinION/examples/tpr_fpr_24barcodes_CPP.csv")
barcode_24 = pd.read_csv("/home/emre/github_repo/MinION/examples/tpr_fpr_24barcodes.csv")
barcode_8 = pd.read_csv("/home/emre/github_repo/MinION/examples/tpr_fpr_8barcodes.csv")

barcode_8_cpp["Barcode"] = "8 barcodes (CPP)"
barcode_24_cpp["Barcode"] = "24 barcodes (CPP)"
barcode_24["Barcode"] = "24 barcodes (Guppy)"
barcode_8["Barcode"] = "8 barcodes (Guppy)"

summary = pd.concat([barcode_8_cpp, barcode_24_cpp, barcode_24, barcode_8]).reset_index(drop=True)
summary = summary.drop(summary[(summary["Barcode"] == "8 barcodes (Guppy)") & (summary["Interval"] == "(20, 25]")].index)


In [ ]:
# Dataframes as per your setup
df = summary
df_8 = df[df['Barcode'] == "8 barcodes (CPP)"]
df_24 = df[df['Barcode'] == "24 barcodes (CPP)"]
df_8_guppy = df[df['Barcode'] == "8 barcodes (Guppy)"]
df_24_guppy = df[df['Barcode'] == "24 barcodes (Guppy)"]

# Define bar width and positions
barWidth = 0.2
r1 = np.arange(len(df_8['Interval']))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
r4 = [x + barWidth for x in r3]

# Create the figure and axes
plt.figure(figsize=(10, 6))

# Creating bars
plt.bar(r1, df_8['TPR'], color='navy', width=barWidth, edgecolor='grey', label='8 Barcodes - TPR - Local')
plt.bar(r2, df_24['TPR'], color="lightsteelblue", width=barWidth, edgecolor='grey', label='24 Barcodes - TPR - Local')
plt.bar(r3, df_8_guppy['TPR'], color='forestgreen', width=barWidth, edgecolor='grey', label='8 Barcodes - TPR - Semi-Global')
plt.bar(r4, df_24_guppy['TPR'], color='limegreen', width=barWidth, edgecolor='grey', label='24 Barcodes - TPR - Semi-Global')

# Add xticks on the middle of the group bars
plt.xlabel('Barcode Score Interval', size=16)
plt.ylabel('True Positive Rate (TPR)', size=16)
plt.xticks([r + barWidth for r in range(len(df_8['Interval']))], df_8['Interval'], rotation=45)

# Create legend & Show graphic
plt.legend()
plt.grid(True)
plt.savefig("tpr_fpr_8_24_barcodes_barplot.png", dpi=300)
plt.show()

In [ ]:
summary = analyser.read_summary_file("/home/emre/github_repo/MinION/examples/data/01_quality/demultiplex_30/")

# Calculate precision, recall, F1 score
summary["Truth"] = summary["read_id"].apply(split_read_id)

barcode_classification = {"True Positive": "TP", "False Positive": "FP", "False Negative": "FN", "True Negative": "TN"}
for barcode in barcodes.keys():
    barcode_column = "barcode_arrangement"
    # barcode = RB01
    tp = summary[(summary[barcode_column] == barcode) & (summary["Truth"] == barcode)]
    fn = summary[(summary[barcode_column] != barcode) & (summary["Truth"] == barcode)]
    fp = summary[(summary[barcode_column] == barcode) & (summary["Truth"] != barcode)]
    tn = summary[(summary[barcode_column] != barcode) & (summary["Truth"] != barcode)]

    tp_percentage = len(tp) / 40000 * 100
    fp_percentage = len(fp) / 40000 * 100

    print(f"True Positive: {len(tp)} ({tp_percentage:.2f}%), False Positive: {len(fp)} ({fp_percentage:.2f}%)")

In [ ]:
summary[(summary[barcode_column] == "RB01") & (summary["Truth"] != "RB01")]

In [ ]:
demultiplexer.run_demultiplexer_single(Path("data") / "03_0", BARCODES, 50, 50, basecall_folder = Path("data") / "03_0")

In [ ]:
summary["barcode_full_arrangement"].value_counts().reset_index()

# SNS
fig, ax = plt.subplots(figsize=(10, 5))
plt.bar(summary["barcode_arrangement"].value_counts().index, summary["barcode_arrangement"].value_counts().values, color="lightgrey", edgecolor="black")
#plt.title("Barcode Arrangement - Guppy (Semi-Global)")
plt.xlabel("Barcode Classification")
plt.ylabel("Frequency")



In [ ]:
# Set the aesthetic style of the plots
sns.set_theme(style="whitegrid")

# Create figure and axes
fig, ax = plt.subplots(figsize=(10, 5))

# Create the histogram plot
sns.histplot(data=summary_01, x="barcode_score", bins=100, kde=False, ax=ax, color="navy", alpha=0.5)
sns.histplot(data=summary_02, x="barcode_score", bins=100, kde=False, ax=ax, color="pink", alpha=0.9)
sns.histplot(data=summary_03, x="barcode_score", bins=100, kde=False, ax=ax, color="lightgreen", alpha=0.5)
sns.histplot(data=summary_04, x="barcode_score", bins=100, kde=False, ax=ax, color="red", alpha=0.5)

# Add custom legend with the correct colors
custom_lines = [plt.Line2D([0], [0], color="navy", lw=4),
                plt.Line2D([0], [0], color="pink", lw=4),
                plt.Line2D([0], [0], color="lightgreen", lw=4),
                plt.Line2D([0], [0], color="red", lw=4)]

ax.legend(custom_lines, ['10% ', '20%', "30%", "40%"])
# Add labels and title
ax.set_xlabel("Barcode Score")
ax.set_ylabel("Frequency")
#ax.set_title("Barcode Score Distribution")

# Add a red vertical line
#ax.axvline(x=60, color='red', linestyle='--')

# Show the plot
#plt.savefig("barcode_score_distribution_guppy.png", dpi=300)
plt.show()

In [ ]:
summary = pd.concat([summary_01, summary_02, summary_03, summary_04])

In [ ]:
bins = np.arange(0, 100, 5)
summary["bins"] = pd.cut(summary["barcode_score"], bins)
summary["Truth"] = summary["read_id"].apply(split_read_id)

In [ ]:
summary["barcode_arrangement"].value_counts().reset_index()

In [ ]:
def split_read_id(read_id):
    """
    Split the read ID into the barcode ID and the read ID.
    """
    parts = read_id.split("_")
    barcode_id = parts[-1]
    read_id_prefix = "_".join(parts[:-1])
    return barcode_id


In [ ]:
summary["Truth"] = summary["read_id"].apply(split_read_id)

In [ ]:
def get_correctly_classified_barcodes(summary_df, barcode_truth_mapping):
    """
    Get correctly classified barcodes from the summary dataframe and compile them into a single DataFrame.

    :param summary_df: Pandas DataFrame containing barcode classification results
    :param barcode_truth_mapping: Dictionary where keys are the expected barcode arrangements
                                  and values are the corresponding truth values.
    :return: DataFrame with an additional column 'Barcode' indicating the barcode name
    """
    frames = []
    for arrangement, truth in barcode_truth_mapping.items():
        temp_df = summary_df[(summary_df["barcode_full_arrangement"] == arrangement) & (summary_df["Truth"] == truth)].copy()
        temp_df['Barcode'] = truth
        frames.append(temp_df)
    
    return pd.concat(frames)

# Example usage
barcode_truth_mapping = {
    "barcode01": "RB01",
    "barcode02": "RB02",
    "barcode03": "RB03",
    "barcode04": "RB04",
    "barcode05": "RB05",
    "barcode06": "RB06",
    "barcode07": "RB07",
    "barcode08": "RB08"
}


In [ ]:
correctly_classified_df = get_correctly_classified_barcodes(summary_01, barcode_truth_mapping)


In [ ]:
correctly_classified_df

In [ ]:
# Correctly classified
rb01 = summary[(summary["barcode_arrangement"] == "barcode01") & (summary["Truth"] == "RB01")]
rb02 = summary[(summary["barcode_arrangement"] == "barcode02") & (summary["Truth"] == "RB02")]
rb03 = summary[(summary["barcode_arrangement"] == "barcode03") & (summary["Truth"] == "RB03")]

In [ ]:
summary_cpp = analyser.read_summary_file(Path('/home/emre/github_repo/MinION/examples/data/Demultiplex_cpp/50'))

In [ ]:
summary_cpp["RBC"].value_counts().reset_index()

# SNS
fig, ax = plt.subplots(figsize=(10, 5))
sns.set_theme(style="whitegrid")
sns.barplot(x="RBC", y="count", data=summary_cpp["RBC"].value_counts().reset_index())
plt.title("Barcode Arrangement - Ours")
plt.xlabel("Barcode Classification")
plt.ylabel("Frequency")


In [ ]:
# Set the aesthetic style of the plots
sns.set_theme(style="whitegrid")

# Create figure and axes
fig, ax = plt.subplots(figsize=(10, 5))

# Create the histogram plot
sns.histplot(data=summary_cpp, x="RBC_Score", bins=100, kde=True, ax=ax)

# Add labels and title
ax.set_xlabel("Barcode Score")
ax.set_ylabel("Frequency")
ax.set_title("Barcode Score Distribution")

# Add a red vertical line
ax.axvline(x=60, color='red', linestyle='--')

# Show the plot
plt.show()

# Alignment Simulation to predict the accuracy at different depths

- Start with HETCPII (612bp)
- Sample random mutations (Parent, Single Point Mutation, 2, 3 up to 8)
- Generate Substitutions & indels for a given site

In [ ]:
def generate_variants(template, num_variants, num_mutations, seed):
    """
    Generate random variants based of num of mutations and num of variants.
    """
    trans_matrix = np.array([   [0, 0.01, 0.46, 0.18, 0.01], #A
                                [0.02, 0, 0.025, 0.43, 0.01], #C
                                [0.43, 0.025, 0, 0.02, 0.01], #G
                                [0.18, 0.0335, 0.1, 0, 0.01], #T
                                [0.01, 0.01, 0.01, 0.01, 0]]) #DEL



    bases = ["A", "C", "G", "T", "DEL"]

    Variants = {"Variant": [], "Sequence": []}

    rng = np.random.RandomState(seed)  # Random state object


    for i in range(num_variants):
        if num_mutations == 0:
            Variants["Variant"].append(["#PARENT#"])
            Variants["Sequence"].append(template)
            continue

        positions = rng.choice(range(len(template)), num_mutations, replace=False)
        positions.sort()

        
        mutations = []
        for pos in positions:
            adjusted_pos = pos 
            ref_base = template[adjusted_pos]
            ref_index = bases.index(ref_base)

            prob = trans_matrix[ref_index]
            prob = prob / prob.sum()

            new_base = np.random.choice(bases, p=prob)

            if new_base != "DEL":
                mutation = f"{ref_base}{adjusted_pos + 1}{new_base}"
            elif new_base == "DEL":
                mutation = f"{ref_base}{adjusted_pos + 1}DEL"
                
            else:
                raise ValueError("Invalid base")
                

            mutations.append((adjusted_pos, new_base, mutation))

        variant = []
        mutated_sequence = list(template)
        for pos, new_base, mut in mutations:
            variant.append(mut)
            if new_base != "DEL":
                mutated_sequence[pos] = new_base
            else:
                # Replace the base with a placeholder for deletion
                mutated_sequence[pos] = "_"

        mutated_sequence = "".join(mutated_sequence).replace("_", "")
        Variants["Variant"].append(variant)
        Variants["Sequence"].append(mutated_sequence)

    return pd.DataFrame(Variants)

def select_random_reads(input_file, output_file, num_reads):
    all_records = list(SeqIO.parse(input_file, "fasta"))

    if num_reads > len(all_records):
        print("Requested number of reads is more than available in the file. Selecting all reads.")
        selected_records = all_records
    else:
        selected_records = random.sample(all_records, num_reads)

    with open(output_file, "w") as output_handle:
        SeqIO.write(selected_records, output_handle, "fasta")


def run_alignment_and_indexing(ref, fasta_file, output_dir):
    """
    Aligns sequences using minimap2, converts to BAM, sorts and indexes the BAM file.

    Args:
    ref (str): Path to the reference genome file.
    fasta_file (str): Path to the FASTA file containing reads.
    output_dir (Path or str): Directory to store output files.

    Returns:
    None
    """
    output_dir = Path(output_dir)
    
    # Run Minimap2 for alignment
    minimap_cmd = f"minimap2 -ax map-ont -O 3,24 {ref} {fasta_file} > {output_dir}/alignment_minimap.sam"
    subprocess.run(minimap_cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Convert SAM to BAM
    view_cmd = f"samtools view -bS {output_dir}/alignment_minimap.sam > {output_dir}/alignment_minimap.bam"
    subprocess.run(view_cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Sort BAM file
    sort_cmd = f"samtools sort {output_dir}/alignment_minimap.bam -o {output_dir}/alignment_minimap.bam"
    subprocess.run(sort_cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Index BAM file
    index_cmd = f"samtools index {output_dir}/alignment_minimap.bam"
    subprocess.run(index_cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

def adjust_variant(variant, padding_start):

    if "#PARENT#" in variant:
        return "#PARENT#"
    
    elif variant == "NA":
        return "NA"
    
    else:
        variants = variant.split('_')
        adjusted_variants = []

        for v in variants:
            # Find the position number using regular expression
            match = re.search(r'([A-Za-z]+)(\d+)([A-Za-z]+)', v)
            if match:
                refAA, pos, newAA = match.groups()
                
                adjusted_pos = max(int(pos) - padding_start, 1)  
                adjusted_variants.append(f"{refAA}{adjusted_pos}{newAA}")

    return '_'.join(adjusted_variants)



In [ ]:
seed = 42

template = IO_processor.read_fasta_file(Path("/home/emre/github_repo/MinION/minION/refseq/hetcpiii.fasta"))["Sequence"][0]

Variants = pd.DataFrame({"Variant" : [], "Sequence" : [], "Num_Mutations" : []})


for n_mut in range(0,11):
    var = generate_variants(template, 100, n_mut, seed)
    var["Num_Mutations"] = n_mut
    Variants = pd.concat([Variants, var]).reset_index(drop=True)

Variants.to_pickle("Variants_100_p_s.pkl")

### Simulate mutation prediction of AF analysis and medaka consensus for different # of mutations

In [ ]:
Variants = pd.read_pickle("Variants_100_p_s.pkl")
# Select mutation with 0, 1, 5, 10
variants = Variants[Variants["Num_Mutations"].isin([0, 1, 5, 10])].reset_index(drop=True)


### Run Simulation

- Enter the depths to analyze
- Padding length
- Num Mutations

In [ ]:
from tqdm import tqdm

# Generate substitution and indels for each variant
template = IO_processor.read_fasta_file(Path("/home/emre/github_repo/MinION/minION/refseq/hetcpiii_padded.fasta"))["Sequence"][0]
padding = 50
os.makedirs(f"data/min_read_depth/seq", exist_ok=True)

ref = "/home/emre/github_repo/MinION/minION/refseq/hetcpiii_padded.fasta"
reference = "HetCPIII"

results = {"Original Variant" : [], "Predicted Variant" : [], "Depth" : [], "Num Mutations" : [], "Correct" : [], "Alignment Frequency" : []}

np.random.seed(43)
random.seed(43)

for i, variant in tqdm(Variants.iterrows()):

    max_depth = 50
    
    if "#PARENT#" in variant["Variant"]:
        var_name = f"wt_{i}"

    else:
        #sort variant
        var_name = "_".join(variant["Variant"])

    # Create folder for each variant
    os.makedirs(f"data/min_read_depth/seq/{var_name}", exist_ok=True)

    padding_seq1 = "aattcccctctagaaataattttgtttaactttaagaaggagatatacat"
    padding_seq2 = "gatccggctgctaacaaagcccgaaaggaagctgagttggctgctgccac"

    with open(f"data/min_read_depth/seq/{var_name}/{var_name}.fasta", "w") as handle: # Create fasta file
        for j in range(max_depth):
            new_seq = introduce_mutations(variant["Sequence"], 0.01)

            new_seq = padding_seq1 + new_seq + padding_seq2

            handle.write(f">{var_name}_{j+1}\n")
            handle.write(f"{new_seq}\n")

    #depths = [1,3,5,7,9,11,13,15,17,19,21,23,25,30,35,40,45,50]
    depths = [10,20,30,40,50]
    # depths = [30,40,50]
    for depth in depths:

        #Create depth folder
        depth_folder_path = f"data/min_read_depth/seq/{var_name}/depth_{depth}"
        os.makedirs(depth_folder_path, exist_ok=True)

        # Select random reads from the fasta file, Create first all depth lengths and store all fasta files in a folder 
        select_random_reads(f"data/min_read_depth/seq/{var_name}/{var_name}.fasta", f"{depth_folder_path}/{var_name}_reads.fasta", depth)

        # prompt = f'mini_align -r {ref} -i {depth_folder_path}/*.fasta -t 1 -m -p alignment && mv *.bam *.bam.bai {depth_folder_path}'
        # subprocess.run(prompt, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

        # Run alignment and indexing
        run_alignment_and_indexing(ref, f"{depth_folder_path}/{var_name}_reads.fasta", depth_folder_path)

        bam_file = f"{depth_folder_path}/alignment_minimap.bam"

        # Call Variants with bam file

        if method == "AF":
            variant_pred = analyser.call_variant_pop_frequency(Path(bam_file), template, reference, min_freq=0.1, min_depth= 0, padding_start=padding, padding_end= padding + 1)
            
            try:
                variant_pred = pd.DataFrame(variant_pred).sort_values("Alignment Frequency", ascending=False).reset_index(drop=True)
                variant_pred["Variant"] = variant_pred["Variant"].apply(lambda x: adjust_variant(x, padding))
                result = 1 if variant_pred["Variant"][0] == "_".join(variant["Variant"]) else 0

                results["Original Variant"].append(variant["Variant"])
                results["Predicted Variant"].append(variant_pred["Variant"][0])
                results["Depth"].append(depth)
                results["Num Mutations"].append(len(variant["Variant"]) if "#PARENT#" not in variant["Variant"] else 0)
                results["Correct"].append(result)
                results["Alignment Frequency"].append(variant_pred["Alignment Frequency"][0])
            except:
                result = "NA"
                results["Original Variant"].append(variant["Variant"])
                results["Predicted Variant"].append("NA")
                results["Depth"].append(depth)
                results["Num Mutations"].append(len(variant["Variant"]))
                results["Correct"].append(result)
                results["Alignment Frequency"].append("NA")
        
        elif method == "guppy":

            
            
# Delete seq folder

# if os.path.exists("data/min_read_depth/seq"):
#     shutil.rmtree("data/min_read_depth/seq")


results = pd.DataFrame(results)

#results.to_csv("data/min_read_depth/results_100_p_s_Q10.csv", index=False)
#results.to_pickle("data/min_read_depth/results_100_p_s_Q20.pkl")


In [ ]:
import glob
from tqdm import tqdm
ref_seq = Path("/home/emre/github_repo/MinION/minION/refseq/hetcpiii_padded.fasta")
folders = glob.glob("data/min_read_depth/seq/*")

for var_path in tqdm(folders):
    var_namr = os.path.basename(var_path)
    depths = glob.glob(f"{var_path}/depth*")
    for depth in depths:
        
        folder_path = Path(depth)
        print("Processing", folder_path)
        consensus.get_consensus(folder_path, ref_seq, output_name = "consensus.fastq", qualities = True, consensus_folder = folder_path)


In [ ]:
### Get variant df



In [ ]:
import glob
from tqdm import tqdm
ref_seq = Path("/home/emre/github_repo/MinION/minION/refseq/hetcpiii_padded.fasta")
folders = glob.glob("data/min_read_depth/seq/*")

for var_path in tqdm(folders):
    var_namr = os.path.basename(var_path)
    depths = glob.glob(f"{var_path}/depth*")
    for depth in depths:
        
        folder_path = Path(depth)
        print("Processing", folder_path)

In [ ]:
pd.DataFrame(results).to_pickle("data/min_read_depth/results_100_p_s_Q20_local.pkl")

In [ ]:
df = pd.DataFrame(results)
#df.to_csv("data/min_read_depth/results_20_p_s_01.csv", index=False)
#df.drop(df[df["Correct"] == "NA"].index, inplace=True)
df.tail(30)["Original Variant"].value_counts()
df = df.drop(df[df["Correct"] == "NA"].index).reset_index(drop=True)

In [ ]:
results

In [ ]:
import re


def get_DEL_samples(entry):
    for variant in entry:
        if "DEL" in variant:
            return 1
    return 0

def check_neg_DEL_position(row):
    variant_list = row['Original Variant']
    check_list = row['Predicted Variant'].split("_")

    
    if "#PARENT#" in variant_list:
        return 0

    min_length = min(len(variant_list), len(check_list))
    
    for i in range(min_length):
        var = variant_list[i]
        if "DEL" in var:
            match_orig = re.search(r'([A-Za-z]+)(\d+)([A-Za-z]+)', var)
            match_pred = re.search(r'([A-Za-z]+)(\d+)([A-Za-z]+)', check_list[i])
            
            if match_orig and match_pred:
                pos = int(match_orig.group(2))
                pos_pred = int(match_pred.group(2))
                
                if pos == pos_pred or pos_pred + 1 == pos or pos_pred - 1 == pos:
                    return 1

    return 0
        
        # Additional processing can be added here as needed
df_wo_del = df.copy()
df_wo_del["DEL"] =      df_wo_del["Original Variant"].apply(get_DEL_samples)
df_wo_del["Corr DEL"] = df_wo_del.apply(check_neg_DEL_position, axis=1)
mask = df_wo_del[(df_wo_del["DEL"] ==1) & (df_wo_del["Corr DEL"] == 1) & (df_wo_del["Correct"] == 0)].index
df_wo_del.iloc[mask, 4] = 1

In [ ]:
df_wo_del[(df_wo_del["Correct"] == 0) & (df_wo_del["Depth"] == 50)]

In [ ]:
grouped_stats_wo_del = df_wo_del.groupby(['Num Mutations', 'Depth'])['Correct'].agg(['mean', 'std']).reset_index()
grouped_stats_wo_del

In [ ]:
grouped_stats = df.groupby(['Num Mutations', 'Depth'])['Correct'].agg(['mean', 'std']).reset_index()
grouped_stats

In [ ]:
results["DEL"] = results["Original Variant"].apply(get_DEL_samples)


In [ ]:
#SHow row with Na
from scipy.stats import linregress

#df = pd.DataFrame(results)

#df = df_wo_del

# Assuming df is your DataFrame
df.drop(df[df["Correct"] == "NA"].index, inplace=True)

df = df.sort_values("Num Mutations")
#df['Num Mutations'] = df['Num Mutations'].astype(str)
df['Correct'] = df['Correct'].astype(float)



# Find unique values in 'Num Mutations' and sort them
unique_mutations = sorted(df['Num Mutations'].unique())

# Create a color palette with a color for each unique 'Num Mutations' value
palette = sns.color_palette("viridis", n_colors=len(unique_mutations))

# Map each unique 'Num Mutations' value to a color
color_map = dict(zip(unique_mutations, palette))

# Plotting
plt.figure(figsize=(10, 6))

# Use the lineplot with the custom palette
sns.lineplot(data=df, x="Depth", y="Correct", hue="Num Mutations", style="Num Mutations", 
             markers=True, dashes=False, palette=color_map, linewidth=2, markersize=10)

# Customization
plt.xlabel("Read Depth", size=16)
plt.ylabel("Accuracy", size=16)
plt.xticks(size=14)
plt.yticks(size=14)
# You can uncomment this if you want a legend
# plt.legend(title="# Mutations", title_fontsize=14, fontsize=14)

plt.savefig("depth_vs_correct_Q20.png", dpi=300)
plt.show()

In [ ]:
grouped_stats

In [ ]:
# Plot Alignment Frequency for number of mutations

plt.bar(df["Num Mutations"].unique(), df.groupby("Num Mutations").mean()["Alignment Frequency"])

In [ ]:
df.groupby("Num Mutations").mean()["Correct"]

In [ ]:
template = IO_processor.read_fasta_file(Path("/home/emre/github_repo/MinION/minION/refseq/hetcpiii.fasta"))["Sequence"][0]


Variants = {"Variant" : [], "Sequence" : []}

var = ["A62T", "A224DEL", "A317T", "T348A", "A596G"]
template = list(template)
# Delete base at position 2 and 3
for v in var:
    match = re.search(r'([A-Za-z]+)(\d+)([A-Za-z]+)', v)
    if match:
        refAA, pos, newAA = match.groups()
        template[int(pos) - 1] = ""
Variants["Variant"].append(var)
Variants["Sequence"].append("".join(template))
print(len(Variants["Sequence"][0]))

In [ ]:
template = IO_processor.read_fasta_file(Path("/home/emre/github_repo/MinION/minION/refseq/hetcpiii_padded.fasta"))["Sequence"][0]
padding_start = 50
padding_end = 50
bam_file = "/home/emre/github_repo/MinION/examples/data/min_read_depth/seq/G22A_C54A_C93T_A105G_G178A_A335T_T519C_C607DEL/depth_50/alignment_minimap.bam"
alignment_count = int(subprocess.run(f"samtools view -c {bam_file}", shell=True, capture_output=True).stdout.decode("utf-8").strip())
range_positions = range(padding_start, len(template) - padding_end)
freq_dist = pd.DataFrame(analyser.get_highest_non_ref_base_freq_2(bam_file, reference, range_positions, template, qualities=False)[0]).T.rename(columns={0:"Base", 1:"Frequency"})
nb_positions = analyser.get_nb_positions(freq_dist, 0.4)
freq_df = analyser.get_pop_frequency(bam_file, template, reference, nb_positions, min_freq=0.1, min_depth= 0)
bases_df = analyser.get_bases_from_pileup(bam_file, reference, [655,656,657,658])
bases_df

In [ ]:
bam_file = "/home/emre/github_repo/MinION/examples/data/min_read_depth/seq/A393G/depth_50/alignment.bam"
variant_pred = analyser.call_variant_pop_frequency(bam_file, template, reference, min_freq=0.1, min_depth= 1)

In [ ]:
variant_pred

In [ ]:
alignment_count = int(subprocess.run(f"samtools view -c {bam_file}", shell=True, capture_output=True).stdout.decode("utf-8").strip())
freq_dist = pd.DataFrame(analyser.get_highest_non_ref_base_freq_2(bam_file, reference, range(1,len(template)), template, qualities=False)[0]).T.rename(columns={0:"Base", 1:"Frequency"})

nb_positions = analyser.get_nb_positions(freq_dist, 0.4)

In [ ]:
analyser.get_pop_frequency(bam_file, template, reference, nb_positions, min_freq=0.1, min_depth= 0)

In [ ]:
pd.DataFrame(variant_pred).sort_values("Alignment Frequency", ascending=False).reset_index(drop=True)

In [ ]:
Variant_manual = {"variant" : [["G23A", "C336T", "C587DEL"], ["C283DEL", "C387T", "A478G"], ["G3A", "G123A", "G229A", "C442T"] ]}

In [ ]:
Variants_manual = Variants[Variants["Variant"].isin(Variant_manual["variant"])]

In [ ]:

bam_file = "/home/emre/github_repo/MinION/examples/data/min_read_depth/seq/G3A_G123A_G229A_C442T/depth_50/alignment.bam"
prompt = f"medaka consensus {bam_file} /home/emre/github_repo/MinION/examples/data/min_read_depth/seq/G3A_G123A_G229A_C442T/depth_50/pre_consensus.hdf --batch 200 --threads 4"
subprocess.run(prompt, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

In [ ]:
prompt = f"medaka stitch /home/emre/github_repo/MinION/examples/data/min_read_depth/seq/G3A_G123A_G229A_C442T/depth_50/pre_consensus.hdf {ref} /home/emre/github_repo/MinION/examples/data/min_read_depth/seq/G3A_G123A_G229A_C442T/depth_50/result.fasta --threads 4"

subprocess.run(prompt, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)


In [ ]:
# align ref and result with bioaligner

ref = "/home/emre/github_repo/MinION/minION/refseq/hetcpiii.fasta"
result = "/home/emre/github_repo/MinION/examples/data/min_read_depth/seq/G3A_G123A_G229A_C442T/depth_50/result.fasta"

prompt = f"bioaligner align {ref} {result} --outfmt sam --out /home/emre/github_repo/MinION/examples/data/min_read_depth/seq/G3A_G123A_G229A_C442T/depth_50/alignment.sam"

subprocess.run(prompt, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

In [ ]:
# Padding the reference sequence with 50 Ns at the start and end

# with open("/home/emre/github_repo/MinION/minION/refseq/hetcpiii.fasta", "r") as handle:
#     records = list(SeqIO.parse(handle, "fasta"))
#     template = str(records[0].seq)

# template = "N" * 50 + template + "N" * 50

# with open("/home/emre/github_repo/MinION/minION/refseq/hetcpiii_padded.fasta", "w") as handle:
#     handle.write(f">{records[0].id}\n")
#     handle.write(f"{template}\n")

# Reindexing the reference sequence
prompt = f"samtools faidx /home/emre/github_repo/MinION/minION/refseq/hetcpiii_padded.fasta"
subprocess.run(prompt, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

### Alignment Frequency distribution accross different quality scores

In [ ]:
Variant_df = pd.read_csv("/home/emre/github_repo/MinION/examples/data/min_read_depth/results_100_p_s_Q15.csv")
# Count NaN
Variant_df.dropna(inplace=True) # NA can occur if the pileup analysis fails. For the sake of the simulation, we dropped these rows. (3 Variants out of 1100)


In [ ]:
Variant_df

In [ ]:
def edit_alignment_freq(entry):
    if entry["Alignment Frequency"] == "-":
        return 0
    else:
        return entry["Alignment Frequency"]

Variant_df["Alignment Frequency"] = Variant_df.apply(edit_alignment_freq, axis=1)

In [ ]:
# Assuming Variant_df is your DataFrame and it's already imported
df = Variant_df[Variant_df["Num Mutations"] > 0]
df["Alignment Frequency"] = df["Alignment Frequency"].astype(float)

# Sort the DataFrame numerically first
df = df.sort_values("Num Mutations")

# Group by and calculate mean and SEM
group_stats = df.groupby("Num Mutations")["Alignment Frequency"].agg(['mean', 'std'])

# Now convert "Num Mutations" to string for plotting, after grouping and calculations
group_stats.index = group_stats.index.astype(str)

# Plotting
fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(group_stats.index, group_stats["mean"], yerr=group_stats["std"], color = "lightgrey", edgecolor = "black", capsize=5)
plt.tick_params(size=14, labelsize=14)




# Improving the plot aesthetics
plt.xlabel('# of Mutations', size=18)
plt.ylabel('Alignment Frequency', size=18)
 # Rotate the x-axis labels for better readability
plt.tight_layout()  # Adjust layout to fit everything nicely
plt.savefig("data/min_read_depth/alignment_frequency_vs_num_mutations_Q10.png", dpi=300)
plt.show()